In [4]:
print("Hi")

Hi


In [8]:
import pandas as pd
import numpy as np
import json

ModuleNotFoundError: No module named 'pandas'

In [ ]:
with open('CNH_RUB_T+1__2022-10-20', 'r') as file:
    data_str = file.read()

data_objects = data_str.split('\n')

list_objects = []

for obj in data_objects:
    # Check if obj is not an empty string
    if obj.strip():
        data = json.loads(obj)
        list_objects.append(data)

In [ ]:
d = {'date':[],
     'side':[],
     'id':[],
     'price':[],
     'size':[]}


for obj in list_objects:
    
    if obj['type'] == 'INCREMENT':
        
        if obj['quotes']['added']:
            d['date'].append(obj['date'])
            d['side'].append(obj['side'])
            d['id'].append(obj['quotes']['added'][0]['id'])
            d['price'].append(obj['quotes']['added'][0]['price'])
            d['size'].append(obj['quotes']['added'][0]['size'])
    
        else:
            continue

In [ ]:
df = pd.DataFrame.from_dict(d)
df['date'] = pd.to_datetime(df['date'])
df['price'] = df['price'].astype(float)
df['size'] = df['size'].astype(float)
df['date'] = df['date'].dt.round('1s')
df = df.sort_values(by='date')
df = df.drop('id', axis = 1)
df

,date,side,price,size
0,2022-10-20 07:00:00,BID,8.435,180000.0
423,2022-10-20 07:00:00,ASK,8.470,50000.0
424,2022-10-20 07:00:00,BID,8.440,50000.0
425,2022-10-20 07:00:00,ASK,8.472,50000.0
426,2022-10-20 07:00:00,BID,8.438,50000.0
...,...,...,...,...
159893,2022-10-20 18:59:32,BID,8.421,70000.0
159894,2022-10-20 18:59:35,BID,8.434,110000.0
159895,2022-10-20 18:59:36,BID,8.408,1287000.0
159896,2022-10-20 18:59:37,BID,8.434,110000.0


In [ ]:
# Separate bids and asks, select only 'date', 'price', and 'size'
df_bids = df[df['side'] == 'BID'][['date', 'price', 'size']].copy()
df_asks = df[df['side'] == 'ASK'][['date', 'price', 'size']].copy()

# Sort bids by 'date' (ascending) and 'price' (descending)
df_bids = df_bids.sort_values(by=['date', 'price'], ascending=[True, False])
# Sort asks by 'date' (ascending) and 'price' (ascending)
df_asks = df_asks.sort_values(by=['date', 'price'], ascending=[True, True])

# Define the number of levels
n_levels = 8

# Create an empty DataFrame with unique dates
df_order_book = pd.DataFrame(df['date'].unique(), columns=['date'])

# Pivot bids and asks separately, creating columns for each level
for i in range(1, n_levels+1):
    df_bid_i = df_bids.groupby('date').nth(i-1).reset_index()
    df_bid_i.columns = ['date', f'bid_price_{i}', f'bid_size_{i}']
    
    df_ask_i = df_asks.groupby('date').nth(i-1).reset_index()
    df_ask_i.columns = ['date', f'ask_price_{i}', f'ask_size_{i}']

    # Merge bid and ask level i onto the order book
    df_order_book = df_order_book.merge(df_bid_i, on='date', how='left')
    df_order_book = df_order_book.merge(df_ask_i, on='date', how='left')

# Sort by date
df_order_book = df_order_book.sort_values(by='date')

In [ ]:
df_order_book.fillna(0)

,date,bid_price_1,bid_size_1,ask_price_1,ask_size_1,bid_price_2,bid_size_2,ask_price_2,ask_size_2,bid_price_3,...,ask_price_6,ask_size_6,bid_price_7,bid_size_7,ask_price_7,ask_size_7,bid_price_8,bid_size_8,ask_price_8,ask_size_8
0,2022-10-20 07:00:00,8.460,50000.0,8.464,50000.0,8.459,400000.0,8.465,180000.0,8.458,...,8.466,50000.0,8.455,400000.0,8.466,50000.0,8.454,50000.0,8.467,180000.0
1,2022-10-20 07:00:02,0.000,0.0,8.524,6000.0,0.000,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0
2,2022-10-20 07:00:06,8.458,50000.0,0.000,0.0,8.457,100000.0,0.000,0.0,8.449,...,0.000,0.0,8.448,400000.0,0.000,0.0,8.448,200000.0,0.000,0.0
3,2022-10-20 07:00:07,8.458,50000.0,0.000,0.0,8.457,100000.0,0.000,0.0,8.447,...,0.000,0.0,8.445,50000.0,0.000,0.0,8.445,50000.0,0.000,0.0
4,2022-10-20 07:00:09,8.446,600000.0,0.000,0.0,8.446,400000.0,0.000,0.0,8.446,...,0.000,0.0,8.445,145000.0,0.000,0.0,8.445,145000.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15485,2022-10-20 18:59:32,8.421,70000.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0
15486,2022-10-20 18:59:35,8.434,110000.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0
15487,2022-10-20 18:59:36,8.408,1287000.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0
15488,2022-10-20 18:59:37,8.434,110000.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0
